# Soft Prompt Tuning

Prompt tuning adds a small set of trainable virtual tokens (soft prompts) to the input while keeping the pre-trained model's weight frozen. 
These virtual prompts are not human-readable; they are appended to the start of a prompt to serve as a task-specific guide during LLM training or inferences.

For example, the virtual tokens are prefixed to text for sentiment classification:

```
[virtual tokens] I love Fridays!
```

where `[virtual tokens]` are the inserted embeddings. These virtual tokens can be randomly generated or initialized from a vocabulary.

During training, these token embeddings are updated while the base model remains frozen. 

In [2]:
!pip install datasets==3.6.0

  Using cached datasets-3.6.0-py3-none-any.whl.metadata (19 kB)
Using cached datasets-3.6.0-py3-none-any.whl (491 kB)
  Attempting uninstall: datasets
    Found existing installation: datasets 4.0.0
    Uninstalling datasets-4.0.0:
      Successfully uninstalled datasets-4.0.0


In [2]:
# Import packages
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, default_data_collator, get_linear_schedule_with_warmup
from peft import get_peft_config, get_peft_model, PromptTuningInit, PromptTuningConfig, TaskType, PeftType, PeftConfig, PeftModel
from datasets import load_dataset
from torch.utils.data import DataLoader

from tqdm import tqdm

In [3]:
model_name = "bigscience/bloomz-560m"

username = 'ought/raft'
dataset_name = "twitter_complaints"

In [4]:
# TODO: Load dataset
dataset = load_dataset(username, dataset_name)

In [10]:
# TODO: Display dataset 
print(dataset)
print(dataset['train'].features)
print(dataset['train'][25])

classes = dataset['train'].features['Label'].names
print(classes)

DatasetDict({
    train: Dataset({
        features: ['Tweet text', 'ID', 'Label'],
        num_rows: 50
    })
    test: Dataset({
        features: ['Tweet text', 'ID', 'Label'],
        num_rows: 3399
    })
})
{'Tweet text': Value(dtype='string', id=None), 'ID': Value(dtype='int32', id=None), 'Label': ClassLabel(names=['Unlabeled', 'complaint', 'no complaint'], id=None)}
{'Tweet text': '@nvidiacc I own two gtx 460 in sli. I want to try windows 8 dev preview. Which driver should I use. Can I use the windows 7 one.', 'ID': 25, 'Label': 2}
['Unlabeled', 'complaint', 'no complaint']


In [11]:
# TODO: Add text_label from Label
# Call dataset_enh
dataset_enh = dataset.map(
   lambda x: { 'text_label': [ classes[l] for l in x['Label']] },
   batched=True,
   num_proc=1
)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/3399 [00:00<?, ? examples/s]

In [12]:
print(dataset_enh['train'][4])

{'Tweet text': 'Couples wallpaper, so cute. :) #BrothersAtHome', 'ID': 4, 'Label': 2, 'text_label': 'no complaint'}


In [13]:
# TODO: Load tokenizer
# call variable name tokenizer
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [14]:
# Utility function to preprocess teweets

def preprocess_function(examples):
    text_column = 'Tweet text'
    label_column = 'text_label'
    max_length = 64
    batch_size = len(examples[text_column])
    inputs = [f"{text_column} : {x} Label : " for x in examples[text_column]]
    targets = [str(x) for x in examples[label_column]]
    model_inputs = tokenizer(inputs)
    labels = tokenizer(targets)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i] + [tokenizer.pad_token_id]
        # print(i, sample_input_ids, label_input_ids)
        model_inputs["input_ids"][i] = sample_input_ids + label_input_ids
        labels["input_ids"][i] = [-100] * len(sample_input_ids) + label_input_ids
        model_inputs["attention_mask"][i] = [1] * len(model_inputs["input_ids"][i])
    # print(model_inputs)
    for i in range(batch_size):
        sample_input_ids = model_inputs["input_ids"][i]
        label_input_ids = labels["input_ids"][i]
        model_inputs["input_ids"][i] = [tokenizer.pad_token_id] * (
            max_length - len(sample_input_ids)
        ) + sample_input_ids
        model_inputs["attention_mask"][i] = [0] * (max_length - len(sample_input_ids)) + model_inputs[
            "attention_mask"
        ][i]
        labels["input_ids"][i] = [-100] * (max_length - len(sample_input_ids)) + label_input_ids
        model_inputs["input_ids"][i] = torch.tensor(model_inputs["input_ids"][i][:max_length])
        model_inputs["attention_mask"][i] = torch.tensor(model_inputs["attention_mask"][i][:max_length])
        labels["input_ids"][i] = torch.tensor(labels["input_ids"][i][:max_length])
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

In [15]:
# TODO: Preprocess the Tweets
# Call preprocessed tweets dataset_processed
dataset_processed = dataset_enh.map(
   preprocess_function,
   batched=True
)

Map:   0%|          | 0/50 [00:00<?, ? examples/s]

Map:   0%|          | 0/3399 [00:00<?, ? examples/s]

In [16]:
print(dataset_processed['train'][4])

{'Tweet text': 'Couples wallpaper, so cute. :) #BrothersAtHome', 'ID': 4, 'Label': 2, 'text_label': 'no complaint', 'input_ids': [3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 227985, 5484, 915, 35673, 8562, 29826, 102530, 15, 1427, 207595, 17, 915, 12, 2550, 81623, 14282, 5715, 37623, 77658, 915, 210, 1936, 106863, 3], 'attention_mask': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], 'labels': [-100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, -100, 1936, 106863, 3

In [ ]:
# TODO: Explore preprocessed Tweets


In [17]:
# TODO: Create train and evaluation dataset 
# Must call train set DataLoader loader_train
# Must call test set DataLoader loader_test

batch_size = 8

dataset_train = dataset_processed['test']
dataset_test = dataset_processed['train']


In [19]:
dec_text = tokenizer.decode(dataset_train[0]['input_ids'], skip_special_tokens=True)
print(dec_text)

Tweet text : @BurberryService Thanks for sending my Christmas present with the security protection still on it! pic.twitter.com/iI0DUxOUU2 Label : Unlabeled


## Soft Prompt 

In [20]:
# TODO: Create PEFT configuration
# randomly initialized
config = PromptTuningConfig(
   task_type=TaskType.CAUSAL_LM,
   prompt_tuning_init=PromptTuningInit.RANDOM,
   num_virtual_tokens=8,
   tokenizer_name_or_path=model_name
)

# initialized from a text
config = PromptTuningConfig(
   task_type=TaskType.CAUSAL_LM,
   prompt_tuning_init=PromptTuningInit.TEXT,
   prompt_tuning_init_text="Auto-Panoptic: Cooperative Multi-Component Architecture Search for Panoptic Segmentation.", 
   num_virtual_tokens=8,
   tokenizer_name_or_path=model_name
)


In [21]:
# TODO: Create model for training
base_model = AutoModelForCausalLM.from_pretrained(model_name)

peft_model = get_peft_model(base_model, peft_config=config)

In [22]:
print(peft_model.print_trainable_parameters())

trainable params: 8,192 || all params: 559,222,784 || trainable%: 0.0015
None


In [ ]:
# Create optimizer
lr = 3e-2
num_epochs = 1
optimizer = torch.optim.AdamW(model.parameters(), lr=lr)
lr_scheduler = get_linear_schedule_with_warmup(
   optimizer=optimizer,
   num_warmup_steps=0,
   num_training_steps=(len(loader_train) * num_epochs)
)

In [ ]:
# Train model
# default to CPU
device = "cuda"
device = "cpu"

model.to(device)

for epoch in range(num_epochs):
    model.train()
    total_loss = 0
    for step, batch in enumerate(tqdm(loader_train)):
        batch = {k: v.to(device) for k, v in batch.items()}
        outputs = model(**batch)
        loss = outputs.loss
        total_loss += loss.detach().float()
        loss.requires_grad = True
        loss.backward()
        optimizer.step()
        lr_scheduler.step()
        optimizer.zero_grad()

    model.eval()
    eval_loss = 0
    eval_preds = []
    for step, batch in enumerate(tqdm(loader_test)):
        batch = {k: v.to(device) for k, v in batch.items()}
        with torch.no_grad():
            outputs = model(**batch)
        loss = outputs.loss
        eval_loss += loss.detach().float()
        eval_preds.extend(
            tokenizer.batch_decode(torch.argmax(outputs.logits, -1).detach().cpu().numpy(), skip_special_tokens=True)
        )

    eval_epoch_loss = eval_loss / len(loader_test)
    eval_ppl = torch.exp(eval_epoch_loss)
    train_epoch_loss = total_loss / len(loader_train)
    train_ppl = torch.exp(train_epoch_loss)
    print(f"{epoch=}: {train_ppl=} {train_epoch_loss=} {eval_ppl=} {eval_epoch_loss=}")

## Inferencing

In [ ]:
peft_model_id = peft_model_id = "stevhliu/bloomz-560m_PROMPT_TUNING_CAUSAL_LM"

In [ ]:
## TODO: Load trained model



In [ ]:
## TODO: Write and encode text

input_text = "Tweet text: @uni I hate the food in your canteen. Label: " 
input_text = "Tweet text: @husband The pipe has been leaking for over one month. Can fix this NOW? Label: "



In [ ]:
## TODO: Determine sentiment from model

